In [ ]:
# Max Gadeberg
# Aug 29, 2024

import csv
import requests
import socket
import subprocess

# ServiceNow API details
url = "https://exampledev.service-now.com/api/now/table/cmdb_ci_service"
query = "operational_statusIN1"

# Authentication headers
headers = {
    'Authorization': '',
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

# Output CSV file name
output_file = 'report.csv'

params = {
    'sysparm_fields': 'u_cpe_dns,name,sys_id,operational_status,u_nn_commit_rate,circuitID',
    'sysparm_query': query
}

response = requests.get(url, headers=headers, params=params)
if response.status_code != 200:
    print(f"Failed to fetch data: {response.status_code} - {response.text}")
    exit()

data = response.json().get('result', [])


def check_ping(ip_address):
    try:
        subprocess.check_output(['ping', '-c', '1', ip_address], timeout=2)
        return True
    except subprocess.CalledProcessError:
        return False
    except subprocess.TimeoutExpired:
        return False


def get_ip_address(hostname):
    try:
        ip_address = socket.gethostbyname(hostname)
        return ip_address
    except socket.gaierror as e:
        print(f"Error getting IP address for {hostname}: {e}")
        return None
    except UnicodeError as e:
        print(f"UnicodeError: {e}")
        return None


with open(output_file, 'w', newline='') as csvfile:
    fieldnames = ['u_cpe_dns', 'customer', 'circuitID', 'ip_address', 'pingable']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for record in data:
        print(record)
        sys_id = record.get('sys_id')
        u_cpe_dns = record.get('u_cpe_dns')
        customer = record.get('name')
        circuitID = record.get('name')

        # Perform DNS lookup
        ip_address = get_ip_address(u_cpe_dns)
        if ip_address:
            # Check if IP address is pingable
            pingable = check_ping(ip_address)
        else:
            pingable = False

        # Write to CSV
        writer.writerow({
            'u_cpe_dns': u_cpe_dns,
            'customer': customer,
            'circuitID': circuitID,
            'ip_address': ip_address,
            'pingable': pingable
        })

print(f"Report generated successfully: {output_file}")
